In [ ]:
# configs para graficos
import matplotlib.pyplot as plt
import seaborn as sns
newfigparams = {
                "figure.figsize": (10,4),
                "figure.dpi": 150,
                "axes.titlesize": 20,
                "axes.labelsize": 11,
                "ytick.labelsize": 11,
                "xtick.labelsize": 11,
                "legend.fontsize": 12,
                "legend.frameon": True,
                "lines.linewidth": 1
                }
plt.rcParams.update(newfigparams)
plt.style.use("dark_background")

# cargar el dataset desde un CSV
hipp_df = pd.read_csv("/kaggle/input/hipparcos-star-catalog/hipparcos-voidmain.csv")
# ver la "cabeza", los primeros 5 datos
hipp_df.head(5)

In [ ]:
# Limpieza de datos y creamos la primera forma de distancia
hipp_df_par_corrected = hipp_df[hipp_df["Plx"].notna()] # NaNs

# Paralaje positivo y no cercano a 0
hipp_df_par_corrected = hipp_df_par_corrected[hipp_df_par_corrected.Plx > 0.1] 
hipp_df_par_corrected["d"] = 1/hipp_df_par_corrected["Plx"]
hipp_df_par_corrected["d"].describe()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation

# Function to update the view
def update_view(t, ax, x, y, z, start_angles, end_angles, start_distance, end_distance):
    # Interpolate the angles and distances
    elev = np.interp(t, [0, 1], [start_angles[0], end_angles[0]])
    azim = np.interp(t, [0, 1], [start_angles[1], end_angles[1]])
    distance = np.interp(t, [0, 1], [start_distance, end_distance])

    # Clear the axis
    ax.cla()

    # Set new view parameters
    ax.scatter(x, y, z, c='cyan', s=0.3, marker='o', alpha=0.7)
    ax.set_xlabel("X (cos(Dec) * cos(RA))")
    ax.set_ylabel("Y (cos(Dec) * sin(RA))")
    ax.set_zlabel("Z (sin(Dec))")
    ax.view_init(elev=elev, azim=azim)
    ax.set_title(f"View: Elev={int(elev)}, Azim={int(azim)}, Distance={int(distance)}")

# Main function to create the animation
def create_animation(x, y, z, start_angles=(30, 60), end_angles=(30, 180), 
                     start_distance=100, end_distance=200, frames=100):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Animation function
    ani = animation.FuncAnimation(fig, update_view, frames=np.linspace(0, 1, frames), 
                                  fargs=(ax, x, y, z, start_angles, end_angles, 
                                         start_distance, end_distance), interval=50)

    # Save the animation (optional)
    ani.save('star_animation.mp4', writer='ffmpeg')

    # Show the plot
    plt.show()

# Example usage
ra_rad = np.deg2rad(hipp_df_par_corrected["RAdeg"])
dec_rad = np.deg2rad(hipp_df_par_corrected["DEdeg"])
distance = 1000 / hipp_df_par_corrected["Parallax"]

# Convert to Cartesian coordinates
x = distance * np.cos(dec_rad) * np.cos(ra_rad)
y = distance * np.cos(dec_rad) * np.sin(ra_rad)
z = distance * np.sin(dec_rad)

# Call the animation function
create_animation(x, y, z, start_angles=(30, 60), end_angles=(30, 180), start_distance=100, end_distance=200)
